# Imports

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import powerlaw

import os
import pdb
import pickle

import tensorflow as tf
from tensorflow.python.client import device_lib
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.models import Sequential, Model, load_model
from keras.layers import Activation, Dense, Input, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import plot_model, multi_gpu_model, np_utils
from contextlib import redirect_stdout

# Functions for predicting the mean, standard deviation of 2D Gaussian

## predict() -- generate new Gaussians, apply model, plot results

In [3]:
def predict(model):
    # Make 1000 new Gaussians to apply the model to
    predX, predy = make_gaussians(1000, add_noise=True)
        
    # Apply the model to get predicted means and sigmas of the Gaussians
    pamp, pxmu, pxsig, pymu, pysig = model.predict(predX, batch_size=None, verbose=0)
    
    # Check distribution of difference between true and predicted means, sigmas
    plt.subplot(511)
    _, _, _ = plt.hist(predy[:, 0] - pamp.flatten(), bins=30)
    plt.subplot(512)
    _, _, _ = plt.hist(predy[:, 1] - pxmu.flatten(), bins=30)
    plt.subplot(513)
    _, _, _ = plt.hist(predy[:, 2] - pxsig.flatten(), bins=30)
    plt.subplot(514)
    _, _, _ = plt.hist(predy[:, 3] - pymu.flatten(), bins=30)
    plt.subplot(515)
    _, _, _ = plt.hist(predy[:, 4] - pysig.flatten(), bins=30)
    
    plt.show()
    
    # Check the relation between true and predicted means, sigmas
    oto_amp = np.linspace(1., 10., 32) #one-to-one relation for amplitudes
    oto_means = np.linspace(-1., 1., 32) # one-to-one relation for means
    oto_sigmas = np.linspace(0.25, 4.0, 32) # one-to-one relation for sigmas
    
    fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize=(15, 4))
    
    # Plot the true y's and predicted y's from the NN model
    ax1.scatter(predy[:, 0], pamp.flatten(), marker='.')
    # Plot the 1-to-1 line
    ax1.plot(oto_amp, oto_amp, color='black', ls='--')
    ax1.set_xlabel('True value')
    ax1.set_ylabel('Predicted value')
    ax1.set_title(r'Gaussian Amplitude')

    ax2.scatter(predy[:, 1], pxmu.flatten(), marker='.')
    ax2.plot(oto_means, oto_means, color='black', ls='--')
    ax2.set_xlabel('True value')
    ax2.set_ylabel('Predicted value')
    ax2.set_title(r'Gaussian x-$\mu$')
    
    ax3.scatter(predy[:, 2], pxsig.flatten(), marker='.')
    ax3.plot(oto_sigmas, oto_sigmas, color='black', ls='--')
    ax3.set_xlabel('True value')
    ax3.set_ylabel('Predicted value')
    ax3.set_title(r'Gaussian x-$\sigma$')
    
    ax4.scatter(predy[:, 3], pymu.flatten(), marker='.')
    ax4.plot(oto_means, oto_means, color='black', ls='--')
    ax4.set_xlabel('True value')
    ax4.set_ylabel('Predicted value')
    ax4.set_title(r'Gaussian y-$\mu$')
    
    ax5.scatter(predy[:, 4], pysig.flatten(), marker='.')
    ax5.plot(oto_sigmas, oto_sigmas, color='black', ls='--')
    ax5.set_xlabel('True value')
    ax5.set_ylabel('Predicted value')
    ax5.set_title(r'Gaussian y-$\sigma$')

    fig.show()

## Generate X_train, y_train, X_test, y_test

### Generate 2D Gaussians given mus, sigmas

In [4]:
# Calculate a 2d Gaussian given its mean and standard deviation
def gaussian2d(x_vals, y_vals, amp, x_mu, x_sigma, y_mu, y_sigma):
    return amp * np.exp(-0.5 * ( ((x_vals - x_mu)/x_sigma)**2 + ((y_vals - y_mu)/y_sigma)**2 ))

# Make array that describes Gaussian
def make_gaussians(num, amp_min=1., amp_max = 10.0, mu_min=-1.0, mu_max=1.0, sig_min=0.25, sig_max=4.0, add_noise=False): 

    # Random Gaussian distribution
    #amp = np.random.uniform(amp_min, amp_max, num)
    
    # Power law distribution
    pwr_law = powerlaw.rvs(4, size=num)
    amp = amp_max * pwr_law
    
    # 'Classification': if pwr_law <= 0.5 --> 'XMP' = 1.
    classification = np.zeros(num)
    classification[np.where(pwr_law <= 0.5)[0]] = 1.
    #classification = np_utils.to_categorical(classification)
    
    x_mus = np.random.uniform(mu_min, mu_max, num)
    #x_sigmas = np.random.uniform(sig_min, sig_max, num)
    x_sigmas = pwr_law**2 * sig_max
    
    y_mus = np.random.uniform(mu_min, mu_max, num)
    #y_sigmas = np.random.uniform(sig_min, sig_max, num)
    y_sigmas = pwr_law**3

    x_vals = np.linspace(-10.0, 10.0, 32)
    y_vals = np.linspace(-10.0, 10.0, 32)
    
    x_grid, y_grid = np.meshgrid(x_vals, y_vals)
    
    models = np.zeros((num, 32, 32))
    noise = np.zeros((num, 32, 32))
    noise_stds = np.random.uniform(0.0, sig_max, num)
    
    #if add_noise:
        # Add some noise -- currently noise is Gaussian but all with same standard deviation = 1.0
    #    noise = np.random.normal(0.0, 1.0, models.shape)

    for i in range(num):
        models[i] = gaussian2d(x_grid, y_grid, amp[i], x_mus[i], x_sigmas[i], y_mus[i], y_sigmas[i])
        
        if add_noise:
            noise[i] = np.random.normal(0.0, noise_stds[i], models[0].shape)
    
    # Also want to save and return the true means, sigmas used for the Gaussians
    targets = np.vstack((amp, x_mus, x_sigmas, y_mus, y_sigmas)).T
    
    models = models.reshape(num, 32, 32, 1)
    noise = noise.reshape(num, 32, 32, 1)

    return models + noise, targets, classification

### generate_dataset() -- wrapper to make n_train, n_test Gaussians and reshape

In [5]:
# Create training and test sets
def generate_dataset(n_train=10000, n_test=1000):
    X_train, y_train, class_train = make_gaussians(n_train, add_noise=True)
    X_test, y_test, class_test = make_gaussians(n_test, add_noise=True)
    
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
    
    return X_train, y_train, X_test, y_test, class_train, class_test

## hyperparams() -- generate random hyperparameters

In [6]:
def hyperparam(mnum):
    '''
    Generate a random set of hyperparameters
    
    Input
    -----
    mnum (int) : model index number
    
    Returns
    -------
    hyperpar : dict
    
    '''
    # Define all of the allowed parameter space
    allowed_hpars = dict(learning_rate      = [0.0005, 0.0007, 0.0010, 0.0030, 0.0050, 0.0070, 0.0100],
                         lr_decay           = [0.0, 1.0],
                         num_epochs         = [30, 50, 80],
                         batch_size         = [16, 32, 64, 128], #100, 500, 1000, 2000, 5000],

                         # Number of filters in each convolutional layer
                         conv_filter_1 = [4, 8, 16, 32], #64], 
                         conv_filter_2 = [2, 4, 8, 16], #32, 64], 
                         
                         # Kernel size
                         conv_kernel_1 = [2, 3, 4, 5],
                         conv_kernel_2 = [1, 2, 3, 4],
                         
                         # Stride of each kernal
                         #conv_stride_1 = [1, 2, 4, 6],
                         #conv_stride_2 = [1, 2, 4, 6],
                         #conv_stride_3 = [1, 2, 4, 6],
                         
                         # Pooling kernel size
                         pool_kernel_1 = [2, 3, 4, 5],

                         # Pooling stride
                         #pool_stride_1 = [1, 2, 3],
                         #pool_stride_2 = [1, 2, 3],
                         #pool_stride_3 = [1, 2, 3],
                         
                         # Fully connected layers
                         fc1_neurons   = [64, 128, 256, 512],
                         )
    
    # Generate dictionary of values
    hyperpar = dict({})
    for key in allowed_hpars.keys():
        hyperpar[key] = np.random.choice(allowed_hpars[key])
    
    print ('\t Hyperparameters:')
    print (hyperpar)
    
    # Save these parameters and return the hyperpar
    save_obj(hyperpar, 'model_{0:03d}_hyperparams'.format(mnum))
    print ('\t Saved hyperparameters!')
    
    return hyperpar

## build_model() -- given random hyperparameters, build the model

In [7]:
def build_model(hyperpar):
    # Extract parameters from hyperparameter set
    conv1_filter = hyperpar['conv_filter_1']
    conv2_filter = hyperpar['conv_filter_2']
    
    conv1_kernel = hyperpar['conv_kernel_1']
    conv2_kernel = hyperpar['conv_kernel_2']

    pool1_kernel = hyperpar['pool_kernel_1']
    
    fc1_neurons = hyperpar['fc1_neurons']

    # Build model
    print ('\t Building model from hyperparameters...')
    input1 = Input(shape=(32, 32, 1))#X_train.shape[1], X_train.shape[2], X_train.shape[3])) # this returns the 32x32 2d array of Gaussian
    
    conv1 = Conv2D(filters=conv1_filter, kernel_size=(conv1_kernel, conv1_kernel), activation='relu')(input1)
    conv2 = Conv2D(filters=conv2_filter, kernel_size=(conv2_kernel, conv2_kernel), activation='relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(pool1_kernel, pool1_kernel))(conv2)    
    
    flat1 = Flatten()(pool1)
    fc1 = Dense(fc1_neurons, activation='relu')(flat1)
    
    out1 = Dense(1, activation='linear', name='amplitude')(fc1)
    out2 = Dense(1, activation='linear', name='x_mean')(fc1)
    out3 = Dense(1, activation='linear', name='x_sigma')(fc1)
    out4 = Dense(1, activation='linear', name='y_mean')(fc1)
    out5 = Dense(1, activation='linear', name='y_sigma')(fc1)
    out6 = Dense(1, activation='sigmoid', name='classification')(fc1)
    
    model = Model(inputs=input1, outputs=[out1, out2, out3, out4, out5, out6])
    #model.summary()

    return model

### Create, compile, fit, and evaluate NN model

## evalute_model() -- read in randomized hyperparameters to compile and fit model

For all struggles with h5py saving models: https://stackoverflow.com/questions/60917467/alternating-errors-using-hdf5-library-and-h5py-module

In [14]:
# fit and evaluate a model
def evaluate_model(X_train, y_train, X_test, y_test, class_train, class_test, hyperpar, mnum, verbose=1):
    filepath = os.getcwd() + '/' #os.path.dirname(os.path.abspath(__file__))
    model_name = 'model_{0:03d}'.format(mnum)
    print ('\t Creating model', model_name)

    # Construct neural network, depending on GPUs
    ngpus = len(get_available_gpus())
    if ngpus > 1:
        model = build_model(hyperpar)
        # Make this work on multiple GPUs
        gpumodel = multi_gpu_model(model, gpus=ngpus)
    else:
        gpumodel = build_model(hyperpar)

    # Summarize layers
    summary = False
    if summary:
        with open(filepath + model_name + '.summary', 'w') as f:
            with redirect_stdout(f):
                model.summary()
    # Plot graph
    plotit = False
    if plotit:
        pngname = filepath + model_name + '.png'
        plot_model(model, to_file=pngname)

    # Compile model
    decay = hyperpar['lr_decay']*hyperpar['learning_rate'] / hyperpar['num_epochs']
    optadam = Adam(lr=hyperpar['learning_rate'], decay=decay)

    gpumodel.compile(loss=['mse', 'mse', 'mse', 'mse', 'mse', 'binary_crossentropy'], optimizer=optadam)#, metrics=['mean_squared_error'])

    # Initialise callbacks
    ckp_name = filepath + model_name + '.hdf5'
    sav_name = filepath + model_name + '_save.hdf5'
    csv_name = filepath + model_name + '.log'
    checkpointer = ModelCheckpoint(filepath=ckp_name, verbose=1, save_best_only=True)
    csv_logger = CSVLogger(csv_name, append=True)

    # Fit network
    gpumodel.fit(X_train, [y_train[:,0], y_train[:,1], y_train[:,2], y_train[:,3], y_train[:,4], class_train],
        epochs=hyperpar['num_epochs'], verbose=verbose)

    gpumodel.save(sav_name)
    print ('\t Saved model ' + sav_name)
    
    # Evaluate model
    accuracy = gpumodel.evaluate(X_test, [y_test[:,0], y_test[:,1], y_test[:,2], y_test[:,3], y_test[:,4], class_test], 
        batch_size=hyperpar['batch_size'], verbose=0)

    return accuracy, gpumodel.metrics_names

In [15]:
def restart_model(model_name, X_train, y_train, X_test, y_test, class_train, class_test, epochs=50, batch_size=16, verbose=1):
    # Load model    
    filepath = os.getcwd() + '/' #os.path.dirname(os.path.abspath(__file__))+'/'
    loadname = filepath + model_name + '.hdf5'
    
    print ('\t Loading model...')
    model = load_model(loadname, compile=False)

    # Make this work on multiple GPUs
    gpumodel = model #multi_gpu_model(model, gpus=4)

    gpumodel.compile(loss=['mse', 'mse', 'mse', 'mse', 'mse', 'binary_crossentropy'], optimizer='adam')#, metrics=['mean_squared_error'])
    
    # Initialise callbacks
    ckp_name = filepath + model_name + '_chkp_restart.hdf5'
    sav_name = filepath + model_name + '_save_restart.hdf5'
    csv_name = filepath + model_name + '_restart.log'
    checkpointer = ModelCheckpoint(filepath=ckp_name, verbose=1, save_best_only=True)
    csv_logger = CSVLogger(csv_name, append=True)
    
    # Fit network
    print ('\t Re-fitting model')
    gpumodel.fit(X_train, [y_train[:,0], y_train[:,1], y_train[:,2], y_train[:,3], y_train[:,4], class_train],
        epochs=epochs, verbose=verbose)

    print ('\t Saving model restart')
    gpumodel.save(sav_name)

    # Evaluate model
    print ('\t Evaluating model')
    accuracy = gpumodel.evaluate(X_test, [y_test[:,0], y_test[:,1], y_test[:,2], y_test[:,3], y_test[:,4], class_test], 
        batch_size=batch_size, verbose=0)
    
    return accuracy, gpumodel.metrics_names

In [16]:
# If relevant, check accuracy of the model
# summarize scores
def summarize_results(scores):
    keys = scores.keys()
    
    for ii in keys:
        m, s = np.mean(scores[ii]), np.std(scores[ii])
        
        print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [17]:
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

# Save and load model
def save_obj(obj, dirname):
    with open(dirname + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
def load_obj(dirname):
    with open(dirname + '.pkl', 'rb') as f:
        return pickle.load(f)

In [18]:
# Detect features in a dataset
def localise_features(mnum, repeats=3, restart=False):
    # Generate hyperparameters
    if not restart:
        hyperpar = hyperparam(mnum)
        print ('\t Generated hyperparameters')

    # Generate data
    X_train, y_train, X_test, y_test, class_train, class_test = generate_dataset()
    print ('\t Generated dataset')
    
    # repeat experiment
    allscores = dict({})
    
    for r in range(repeats):
        if restart:
            model_name = 'model_002_save'
            print ('\t Restarting model ', model_name)
            scores, names = restart_model(model_name, X_train, y_train, X_test, y_test, class_train, class_test)
        else:
            scores, names = evaluate_model(X_train, y_train, X_test, y_test, class_train, class_test, hyperpar, mnum)
        if r == 0:
            for name in names:
                #pdb.set_trace()
                allscores[name] = []
        for ii, name in enumerate(names):
            #pdb.set_trace()
            allscores[name].append(scores[ii] * 100.0)
            if '_acc' in name:
                print('%s >#%d: %.3f' % (name, r + 1, allscores[name][-1]))
            else:
                print('%s >#%d: %.3f' % (name, r + 1, scores[ii]))
                
    # Summarize results
    #summarize_results(allscores)

In [19]:
# Set the number of epochs
if False:
    # Generate data
    generate_dataset()
else:
    # Once the data exist, run the experiment
    m_init = 0
    # mnum sets the number of model fitting + evaluations using *different* sets of hyperparameters
    mnum = m_init
    while True:
        try:
            print ('\n \n', mnum)
            # repeats sets the number of model fitting + evaluations using the same set of hyperparameters
            # but with different initial starting points/values for the trainable parameters (weights, biases, etc.)
            localise_features(mnum, repeats=1, restart=False)
            # use restart=True when want to rerun a specific model
            #localise_features(mnum, repeats=3, restart=True)
        except ValueError:
            continue
        mnum += 1
        if mnum >= m_init+5:
            break


 
 0
	 Hyperparameters:
{'learning_rate': 0.0007, 'lr_decay': 1.0, 'num_epochs': 50, 'batch_size': 128, 'conv_filter_1': 16, 'conv_filter_2': 8, 'conv_kernel_1': 3, 'conv_kernel_2': 2, 'pool_kernel_1': 2, 'fc1_neurons': 256}
	 Saved hyperparameters!
	 Generated hyperparameters
	 Generated dataset
	 Creating model model_000
	 Building model from hyperparameters...
Epoch 1/50
10000/10000 [==============================] - 6s 623us/step - loss: 3.6687 - amplitude_loss: 2.5686 - x_mean_loss: 0.2848 - x_sigma_loss: 0.3473 - y_mean_loss: 0.1745 - y_sigma_loss: 0.1161 - classification_loss: 0.1730
Epoch 2/50
10000/10000 [==============================] - 6s 615us/step - loss: 0.8342 - amplitude_loss: 0.3812 - x_mean_loss: 0.1534 - x_sigma_loss: 0.0754 - y_mean_loss: 0.0745 - y_sigma_loss: 0.0250 - classification_loss: 0.1249
Epoch 3/50
10000/10000 [==============================] - 6s 604us/step - loss: 0.6686 - amplitude_loss: 0.3067 - x_mean_loss: 0.1317 - x_sigma_loss: 0.0560 - y_mean_los

10000/10000 [==============================] - 8s 813us/step - loss: 0.0587 - amplitude_loss: 0.0257 - x_mean_loss: 0.0133 - x_sigma_loss: 0.0059 - y_mean_loss: 0.0078 - y_sigma_loss: 8.5032e-04 - classification_loss: 0.0053
Epoch 35/50
10000/10000 [==============================] - 7s 726us/step - loss: 0.0516 - amplitude_loss: 0.0217 - x_mean_loss: 0.0127 - x_sigma_loss: 0.0051 - y_mean_loss: 0.0071 - y_sigma_loss: 7.0265e-04 - classification_loss: 0.0044
Epoch 36/50
10000/10000 [==============================] - 6s 638us/step - loss: 0.0480 - amplitude_loss: 0.0190 - x_mean_loss: 0.0125 - x_sigma_loss: 0.0048 - y_mean_loss: 0.0072 - y_sigma_loss: 7.1358e-04 - classification_loss: 0.0038
Epoch 37/50
10000/10000 [==============================] - 5s 537us/step - loss: 0.0493 - amplitude_loss: 0.0206 - x_mean_loss: 0.0129 - x_sigma_loss: 0.0049 - y_mean_loss: 0.0069 - y_sigma_loss: 6.8768e-04 - classification_loss: 0.0036
Epoch 38/50
10000/10000 [==============================] - 6s 59

10000/10000 [==============================] - 5s 471us/step - loss: 0.3565 - amplitude_loss: 0.1303 - x_mean_loss: 0.0994 - x_sigma_loss: 0.0249 - y_mean_loss: 0.0404 - y_sigma_loss: 0.0022 - classification_loss: 0.0597
Epoch 17/50
10000/10000 [==============================] - 5s 505us/step - loss: 0.3449 - amplitude_loss: 0.1222 - x_mean_loss: 0.0978 - x_sigma_loss: 0.0244 - y_mean_loss: 0.0397 - y_sigma_loss: 0.0021 - classification_loss: 0.0587
Epoch 18/50
10000/10000 [==============================] - 5s 467us/step - loss: 0.3426 - amplitude_loss: 0.1219 - x_mean_loss: 0.0994 - x_sigma_loss: 0.0232 - y_mean_loss: 0.0396 - y_sigma_loss: 0.0022 - classification_loss: 0.0570
Epoch 19/50
10000/10000 [==============================] - 5s 475us/step - loss: 0.3314 - amplitude_loss: 0.1187 - x_mean_loss: 0.0954 - x_sigma_loss: 0.0233 - y_mean_loss: 0.0376 - y_sigma_loss: 0.0021 - classification_loss: 0.0541
Epoch 20/50
10000/10000 [==============================] - 6s 601us/step - loss:

loss >#1: 0.591
amplitude_loss >#1: 0.183
x_mean_loss >#1: 0.172
x_sigma_loss >#1: 0.035
y_mean_loss >#1: 0.065
y_sigma_loss >#1: 0.002
classification_loss >#1: 0.129

 
 2
	 Hyperparameters:
{'learning_rate': 0.01, 'lr_decay': 1.0, 'num_epochs': 80, 'batch_size': 128, 'conv_filter_1': 4, 'conv_filter_2': 8, 'conv_kernel_1': 5, 'conv_kernel_2': 2, 'pool_kernel_1': 5, 'fc1_neurons': 256}
	 Saved hyperparameters!
	 Generated hyperparameters
	 Generated dataset
	 Creating model model_002
	 Building model from hyperparameters...
Epoch 1/80
10000/10000 [==============================] - 4s 437us/step - loss: 6.3735 - amplitude_loss: 4.7332 - x_mean_loss: 0.4740 - x_sigma_loss: 0.4306 - y_mean_loss: 0.2522 - y_sigma_loss: 0.3021 - classification_loss: 0.1720
Epoch 2/80
10000/10000 [==============================] - 4s 389us/step - loss: 0.7636 - amplitude_loss: 0.3535 - x_mean_loss: 0.1527 - x_sigma_loss: 0.0615 - y_mean_loss: 0.0782 - y_sigma_loss: 0.0088 - classification_loss: 0.1084
Epoch

10000/10000 [==============================] - 4s 402us/step - loss: 0.3840 - amplitude_loss: 0.1349 - x_mean_loss: 0.1117 - x_sigma_loss: 0.0287 - y_mean_loss: 0.0419 - y_sigma_loss: 0.0026 - classification_loss: 0.0645
Epoch 34/80
10000/10000 [==============================] - 4s 391us/step - loss: 0.3772 - amplitude_loss: 0.1359 - x_mean_loss: 0.1076 - x_sigma_loss: 0.0290 - y_mean_loss: 0.0422 - y_sigma_loss: 0.0026 - classification_loss: 0.0605
Epoch 35/80
10000/10000 [==============================] - 4s 388us/step - loss: 0.3587 - amplitude_loss: 0.1238 - x_mean_loss: 0.1051 - x_sigma_loss: 0.0269 - y_mean_loss: 0.0423 - y_sigma_loss: 0.0024 - classification_loss: 0.0581
Epoch 36/80
10000/10000 [==============================] - 4s 391us/step - loss: 0.3498 - amplitude_loss: 0.1225 - x_mean_loss: 0.1028 - x_sigma_loss: 0.0269 - y_mean_loss: 0.0412 - y_sigma_loss: 0.0023 - classification_loss: 0.0543
Epoch 37/80
10000/10000 [==============================] - 4s 387us/step - loss:

10000/10000 [==============================] - 4s 390us/step - loss: 0.1648 - amplitude_loss: 0.0518 - x_mean_loss: 0.0621 - x_sigma_loss: 0.0149 - y_mean_loss: 0.0232 - y_sigma_loss: 0.0016 - classification_loss: 0.0111
Epoch 69/80
10000/10000 [==============================] - 5s 530us/step - loss: 0.1632 - amplitude_loss: 0.0512 - x_mean_loss: 0.0605 - x_sigma_loss: 0.0149 - y_mean_loss: 0.0236 - y_sigma_loss: 0.0017 - classification_loss: 0.01135s - loss: 0.1478 - amplit
Epoch 70/80
10000/10000 [==============================] - 4s 405us/step - loss: 0.1655 - amplitude_loss: 0.0530 - x_mean_loss: 0.0618 - x_sigma_loss: 0.0149 - y_mean_loss: 0.0232 - y_sigma_loss: 0.0017 - classification_loss: 0.0111
Epoch 71/80
10000/10000 [==============================] - 4s 405us/step - loss: 0.1651 - amplitude_loss: 0.0523 - x_mean_loss: 0.0623 - x_sigma_loss: 0.0149 - y_mean_loss: 0.0232 - y_sigma_loss: 0.0017 - classification_loss: 0.0106
Epoch 72/80
10000/10000 [=============================

10000/10000 [==============================] - 5s 509us/step - loss: 0.2469 - amplitude_loss: 0.0959 - x_mean_loss: 0.0759 - x_sigma_loss: 0.0195 - y_mean_loss: 0.0306 - y_sigma_loss: 0.0023 - classification_loss: 0.0229
Epoch 20/30
10000/10000 [==============================] - 5s 519us/step - loss: 0.2276 - amplitude_loss: 0.0881 - x_mean_loss: 0.0695 - x_sigma_loss: 0.0183 - y_mean_loss: 0.0302 - y_sigma_loss: 0.0019 - classification_loss: 0.0195
Epoch 21/30
10000/10000 [==============================] - 5s 508us/step - loss: 0.2223 - amplitude_loss: 0.0870 - x_mean_loss: 0.0677 - x_sigma_loss: 0.0176 - y_mean_loss: 0.0295 - y_sigma_loss: 0.0018 - classification_loss: 0.0190
Epoch 22/30
10000/10000 [==============================] - 5s 511us/step - loss: 0.2094 - amplitude_loss: 0.0807 - x_mean_loss: 0.0650 - x_sigma_loss: 0.0167 - y_mean_loss: 0.0276 - y_sigma_loss: 0.0019 - classification_loss: 0.0174
Epoch 23/30
10000/10000 [==============================] - 5s 501us/step - loss:

10000/10000 [==============================] - 7s 717us/step - loss: 0.4495 - amplitude_loss: 0.1711 - x_mean_loss: 0.1185 - x_sigma_loss: 0.0336 - y_mean_loss: 0.0549 - y_sigma_loss: 0.0029 - classification_loss: 0.0688
Epoch 22/30
10000/10000 [==============================] - 7s 672us/step - loss: 0.4313 - amplitude_loss: 0.1570 - x_mean_loss: 0.1156 - x_sigma_loss: 0.0315 - y_mean_loss: 0.0555 - y_sigma_loss: 0.0027 - classification_loss: 0.0686
Epoch 23/30
10000/10000 [==============================] - 7s 668us/step - loss: 0.4259 - amplitude_loss: 0.1548 - x_mean_loss: 0.1169 - x_sigma_loss: 0.0316 - y_mean_loss: 0.0527 - y_sigma_loss: 0.0026 - classification_loss: 0.0671
Epoch 24/30
10000/10000 [==============================] - 7s 676us/step - loss: 0.4176 - amplitude_loss: 0.1500 - x_mean_loss: 0.1147 - x_sigma_loss: 0.0305 - y_mean_loss: 0.0551 - y_sigma_loss: 0.0026 - classification_loss: 0.0649
Epoch 25/30
10000/10000 [==============================] - 7s 668us/step - loss:

In [ ]:
model = load_model('model_002_save.hdf5')

In [ ]:
# Separate low-amp vs high-amp Gaussians and plot the above 5 plots to see if there is 
# a larger scatter for true vs predicted in low-amp Gaussians
a, b, _, _, c, _ = generate_dataset()

low_Z = np.where(b[:,0]/10. < 0.5)[0]

print (len(low_Z), len(a))

In [ ]:
pamp, pxmu, pxsig, pymu, pysig, cls = model.predict(a, batch_size=None, verbose=0)

In [ ]:
# Check the relation between true and predicted means, sigmas
oto_amp = np.linspace(1., 10., 32) #one-to-one relation for amplitudes
oto_means = np.linspace(-1., 1., 32) # one-to-one relation for means
oto_sigmas = np.linspace(0.25, 4.0, 32) # one-to-one relation for sigmas

fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize=(15, 4))

# Plot the true y's and predicted y's from the NN model
ax1.scatter(b[:, 0], pamp.flatten(), marker='.')
ax1.scatter(b[:, 0][low_Z], pamp.flatten()[low_Z], marker='.')
# Plot the 1-to-1 line
ax1.plot(oto_amp, oto_amp, color='black', ls='--')
ax1.set_xlabel('True value')
ax1.set_ylabel('Predicted value')
ax1.set_title(r'Gaussian Amplitude')

ax2.scatter(b[:, 1], pxmu.flatten(), marker='.')
ax2.plot(oto_means, oto_means, color='black', ls='--')
ax2.set_xlabel('True value')
ax2.set_ylabel('Predicted value')
ax2.set_title(r'Gaussian x-$\mu$')

ax3.scatter(b[:, 2], pxsig.flatten(), marker='.')
ax3.plot(oto_sigmas, oto_sigmas, color='black', ls='--')
ax3.set_xlabel('True value')
ax3.set_ylabel('Predicted value')
ax3.set_title(r'Gaussian x-$\sigma$')

ax4.scatter(b[:, 3], pymu.flatten(), marker='.')
ax4.plot(oto_means, oto_means, color='black', ls='--')
ax4.set_xlabel('True value')
ax4.set_ylabel('Predicted value')
ax4.set_title(r'Gaussian y-$\mu$')

ax5.scatter(b[:, 4], pysig.flatten(), marker='.')
ax5.plot(oto_sigmas, oto_sigmas, color='black', ls='--')
ax5.set_xlabel('True value')
ax5.set_ylabel('Predicted value')
ax5.set_title(r'Gaussian y-$\sigma$')

fig.show()

In [ ]:
fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize=(15, 3))

ax1.hist(b[:, 0] - pamp.flatten(), density=True, bins=30, alpha=0.5, label='All')
ax1.hist(b[:, 0][low_Z] - pamp.flatten()[low_Z], density=True, bins=30, alpha=0.5, label='Low Z')
ax1.axvline(0.0, ls='--', color='black', alpha=0.5)
ax1.legend()
ax1.set_title(r'Gaussian amplitude')
ax1.set_xlabel('(true - predicted) value')

ax2.hist(b[:, 1] - pxmu.flatten(), density=True, bins=30, alpha=0.5)
ax2.hist(b[:, 1][low_Z] - pxmu.flatten()[low_Z], density=True, bins=30, alpha=0.5)
ax2.axvline(0.0, ls='--', color='black', alpha=0.5)
ax2.set_title(r'Gaussian x-$\mu$')
ax2.set_xlabel('(true - predicted) value')

ax3.hist(b[:, 2] - pxsig.flatten(), density=True, bins=30, alpha=0.5)
ax3.hist(b[:, 2][low_Z] - pxsig.flatten()[low_Z], density=True, bins=30, alpha=0.5)
ax3.axvline(0.0, ls='--', color='black', alpha=0.5)
ax3.set_title(r'Gaussian x-$\sigma$')
ax3.set_xlabel('(true - predicted) value')

ax4.hist(b[:, 3] - pymu.flatten(), density=True, bins=30, alpha=0.5)
ax4.hist(b[:, 3][low_Z] - pymu.flatten()[low_Z], density=True, bins=30, alpha=0.5)
ax4.axvline(0.0, ls='--', color='black', alpha=0.5)
ax4.set_title(r'Gaussian y-$\mu$')
ax4.set_xlabel('(true - predicted) value')

ax5.hist(b[:, 4] - pysig.flatten(), density=True, bins=30, alpha=0.5)
ax5.hist(b[:, 4][low_Z] - pysig.flatten()[low_Z], density=True, bins=30, alpha=0.5)
ax5.axvline(0.0, ls='--', color='black', alpha=0.5)
ax5.set_title(r'Gaussian y-$\sigma$')
ax5.set_xlabel('(true - predicted) value')

fig.show()

In [ ]:
100 - (100*676/10000)

In [ ]:
len(np.where(c - cls.flatten().astype(int) == 1.)[0]), len(np.where(c - cls.flatten().astype(int) == 0.)[0])

# Plot 2D Gaussian

In [ ]:
# Import for 3d plotting
from mpl_toolkits import mplot3d

In [ ]:
# Create x, y grid, and use it to generate a 2d Gaussian
x_vals = np.linspace(-10, 10, 32)
y_vals = np.linspace(-10, 10, 32)

x, y = np.meshgrid(x_vals, y_vals)

z = gaussian2d(x, y, 1, 0, 2, 0, 2)

# Plot 2d Gaussian
fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection='3d')

ax.plot_surface(x, y, z, rstride=1, cstride=2, cmap='winter')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.show()